# 2. Amazon Comprehend

<img src="./images/comprehend.png">

Amazon Comprehend는 Natural Language Processing (NLP) 를 사용하여 문서들의 콘텐츠에서 insights 를 추출하는 서비스입니다.

-----------------------------------
> 1. **Input** : UTF-8 포맷의 text 파일
> 2. **API** : JAVA, .NET, Python
> 3. **제공 기능** (언어 별로 미제공 기능 있음)
>    - **Entities, Key phrases, Language, Sentiments, Topic Modeling**
>    - **Syntax , Custom Classification (한국어 미지원), Custom Entity Recognition (영어만 가능)**  
-----------------------------------

In [108]:
%load_ext autoreload
%autoreload 2

# External Dependencies:
import time
import boto3
import json
import pandas as pd
import tarfile

comprehend = boto3.client('comprehend')
s3 = boto3.resource('s3')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# ...We just retrieve it here:
%store -r
assert bucket_name, "Variable `bucket_name` missing from IPython store"
print(bucket_name)

## bucket_name ='transcribe-comprehend-demo-test-XXX'  ## CloudFormation의 ouput에서 나온 S3Bucket

assert transcript, "Variable `transcript` missing from IPython store"
print(transcript)

못된 기분이 나빠 있는지 정확하게 물어보더라고요 못된 기분이 나빠 있는지 정확히 물어보더라고요 추가 아니에요. 미친 거 아니에요 죄송합니다. 별빛, 식량 따뜻하고 죄송합니다 별빛, 식량 따뜻하고 다시 한번 거기에 교육도 다시 한번 빠지기 전에 같은 날 다시는 가나가와 알겠습니다. 다시 한 번 관리자에게 내용 전달해서 서비스를 진행 하겠습니다. 죄송합니다. 병신 같은 게 무슨 매니저로 고기 한다는 거죠? 다시 한번 거기에 교육도 다시 한번 빠지기 전에 같은 날 다시는 가나가와 알겠습니다. 다시 한 번 관리자에게 내용 전달해서 서비스를 진행 하겠습니다. 죄송합니다. 병신 같은 게 무슨 매니저로 고기 한다는 거죠? 안전하게 살고 있었어요? 겁니다. 이해하고 매우 만족 하지 않았어요. 안전하게 살고 있었어요? 겁니다. 이해하고 매우 만족 하지 않았어요 이 첫날 사람들이 이 첫날 사람들이 모두가 그렇게 멍청한 농가 라고 했어요. 모두가 그렇게 멍청한 농가 라고 했어요 바꿔 봐요 바꿔 봐요 가슴을 졸이게 마주하는 안전요원 어떻게 했는지 추천한다. 가슴을 졸이게 마주하는 안전요원 어떻게 했는지 추천한다 영남은 있겠죠? 안전 시한인 안되지만 만 곳도 있다. 영남은 있겠죠? 안전 시한인 안되지만 만 곳도 있다 진압을 목청 이들도 못 받은 게 진압을 목청 이들도 못 받은 게 착하고 성실한 우리 딸이 상담 드릴 예정입니다. 착하고 성실한 우리 딸이 상담 드릴 예정입니다. 사랑하는 우리 아내가 상담 드릴 예정입니다. 사랑하는 우리 아내가 상담 드릴 예정입니다. 상담원 도 누군가의 소중한 가족이라는 것을 연결하 을 통해 들려주었습니다. 제가 해외 가장 중요한 우리 엄마가 상담원 도 누군가의 소중한 가족이라는 것을 연결하 을 통해 들려주었습니다. 제가 해외 가장 중요한 우리 엄마가 삼 단적인 예정입니다. 잠시만 기다려주세요. 삼 단적인 예정입니다. 잠시만 기다려주세요. 개인고객 되니까 아, 지경입니다. 되니까요. 개인고객 되니까 아, 지경입니다. 되니까요. 만명의 개입니다. 만명의 개

### Detecting the Dominant Language
-----------------------------------
- **Language (한국어 지원)** : 주요 언어명, 100여개 식별 가능 (한국어 지원)

In [135]:
%%time
print('Calling DetectDominantLanguage')
res = comprehend.detect_dominant_language(Text = transcript)
result = res['Languages'][0]
LanguageCode = result['LanguageCode']
print("Language : {}, Score : {} \n\n".format(LanguageCode, result['Score']))

print("[detail result] \n" + json.dumps(res, sort_keys=True, indent=4))

print("End of DetectDominantLanguage\n")

Calling DetectDominantLanguage
Language : ko, Score : 1.0 


[detail result] 
{
    "Languages": [
        {
            "LanguageCode": "ko",
            "Score": 1.0
        }
    ],
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "49",
            "content-type": "application/x-amz-json-1.1",
            "date": "Wed, 04 Mar 2020 12:10:18 GMT",
            "x-amzn-requestid": "093c7ca0-97fb-44a5-8b43-b1afda396850"
        },
        "HTTPStatusCode": 200,
        "RequestId": "093c7ca0-97fb-44a5-8b43-b1afda396850",
        "RetryAttempts": 0
    }
}
End of DetectDominantLanguage

CPU times: user 9.93 ms, sys: 164 µs, total: 10.1 ms
Wall time: 99.7 ms


### Detecting Named Entities 
-----------------------------------
- **Entities (한국어 지원)** : Named Entitiy Recognition 수행
<img src="./images/NER항목.png" width='600'>

In [49]:
print('Calling DetectEntities')
res = comprehend.detect_entities(Text=transcript, LanguageCode=LanguageCode)
list_result =[]
for result in res['Entities']:
    list_result.append([result['Text'], result['Type'], result['Score'], result['BeginOffset'], result['EndOffset']])
df = pd.DataFrame(list_result, columns=['Text', 'Type', 'Score','BeginOffset', 'EndOffset'])
df=df.sort_values(by='Score', ascending=False)
df

Calling DetectEntities


,Text,Type,Score,BeginOffset,EndOffset
17,대륙,LOCATION,0.999997,1219,1221
16,청와대,LOCATION,0.999990,1128,1131
15,청와대,LOCATION,0.999988,1036,1039
7,영남,LOCATION,0.999746,562,564
8,영남,LOCATION,0.999559,592,594
18,대륙,LOCATION,0.998322,1287,1289
21,이백,QUANTITY,0.991279,1387,1389
14,만명의,QUANTITY,0.986302,995,998
13,만명의,QUANTITY,0.974957,985,988
20,이백,QUANTITY,0.940624,1357,1359


### Detecting Key Phrases
-----------------------------------
- **Key phrases (한국어 지원)** : 문서 내 키워드 추출
   - 특정 사물을 설명하는 명사구를 포함하는 문자열 의미
   - 명사구(관사+형용사+명사)와 신뢰 수준을 제공
   - 모든 문서는 동일 언어로 작성되어야 함

In [51]:
print('Calling DetectKeyPhrases')
res = comprehend.detect_key_phrases(Text=transcript, LanguageCode=LanguageCode)
list_result =[]
for result in res['KeyPhrases']:
    list_result.append([result['Text'], result['Score'], result['BeginOffset'], result['EndOffset']])
df = pd.DataFrame(list_result, columns=['Text', 'Score','BeginOffset', 'EndOffset'])
df = df.sort_values(by='Score', ascending=False)
df 

Calling DetectKeyPhrases


,Text,Score,BeginOffset,EndOffset
172,수,0.999998,1435,1436
179,수,0.999988,1483,1484
173,변화 자체,0.999960,1440,1445
180,변화 자체,0.999905,1488,1493
101,제,0.999881,865,866
149,강성 고객,0.999869,1264,1269
147,고객,0.999864,1240,1242
131,고민,0.999825,1124,1126
165,수,0.999761,1381,1382
60,가슴,0.999753,531,533


### Detecting Sentiment
-----------------------------------
- **Sentiments (한국어 지원)** : 긍정, 부정, 중립, 혼합 제공

In [66]:
print('Calling DetectSentiment')

res = comprehend.detect_sentiment(Text=transcript, LanguageCode=LanguageCode)

print("Sentiment : {}  \n Positive : {:0.5f} \n Negative : {:0.5f} \n Neutral : {:0.5f} \n Mixed : {:0.5f} \n ".format(
res['Sentiment'], res['SentimentScore']['Positive'], res['SentimentScore']['Negative'], res['SentimentScore']['Neutral'], res['SentimentScore']['Mixed']))

Calling DetectSentiment
Sentiment : NEGATIVE  
 Positive : 0.05819 
 Negative : 0.57940 
 Neutral : 0.36240 
 Mixed : 0.00001 
 


### Detecting Syntax
-----------------------------------
- **Syntax (한국어 미지원)**
   - 17개의 Part-of-Speech (PoS) 식별
   - ADJ (형용사),ADP (전치사/후치사),ADV (부사),AUX (조동사),NOUN (명사),NUM 등 

In [2]:
# print('Calling DetectSyntax')

# res = comprehend.detect_syntax(Text=transcript, LanguageCode=LanguageCode)
# list_result =[]
# for result in res['SyntaxTokens']:
#     list_result.append([result['Text'], result['TokenId'],result['PartOfSpeech']['Tag'] ,result['PartOfSpeech']['Score'], result['BeginOffset'], result['EndOffset']])
# df = pd.DataFrame(list_result, columns=['Text', 'TokenId', 'Tag', 'Score', 'BeginOffset', 'EndOffset'])
# df = df.sort_values(by='TokenId')
# df 

text = "It is raining today in Seattle"
print('Calling DetectSyntax')
print(json.dumps(comprehend.detect_syntax(Text=text, LanguageCode='en'), sort_keys=True,
 indent=4))
print('End of DetectSyntax\n')

Calling DetectSyntax


NameError: name 'comprehend' is not defined

### Topic Modeling
-----------------------------------
- **Topic Modeling**  
   - 문서 집합에 대한 공통 테마 결정
   - 정치, 스포츠, 엔터테인먼트 등의 주제로 결정
   - 문서 내 텍스트에 대한 별도 주석이 필요 없음
   - LDA(Latent Dirichlet Allocation) 기반 학습 모델
   - 좋은 결과를 얻기 위해서는, 
       - 최소 1,000개 문서 사용
       - 각 문서 길이는 3문장 이상 필요
       - 문서가 주로 숫자 데이터 위주이면 Corpus에서 제거 


In [112]:
JobName = 'XXXXXXX' ## 작업명
topic_modeling_prefix = 'XXXXXXX'  ## S3 내 topic modeling을 위한 documents 저장 위치
topic_modeling_output = 'XXXXXXX'  ## S3 내 topic modeling 결과 위치 

input_s3_url ="s3://{}/{}".format(bucket_name, topic_modeling_prefix)
input_doc_format = "ONE_DOC_PER_FILE" ## 
output_s3_url = "s3://{}/{}".format(bucket_name, topic_modeling_output)
data_access_role_arn = "arn:aws:iam::XXXXXXXXXX:role/service-role/XXXXXXXXX-DataAccessRole-XXXXXXXXXXX"  ## CloudFormation의 ouput에서 나온 DataAccessRoleArn
number_of_topics = 10

In [ ]:
%%time
input_data_config = {"S3Uri": input_s3_url, "InputFormat": input_doc_format}
output_data_config = {"S3Uri": output_s3_url}

start_topics_detection_job_result = comprehend.start_topics_detection_job(
    JobName=JobName,
    NumberOfTopics=number_of_topics,
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    DataAccessRoleArn=data_access_role_arn)

print('start_topics_detection_job_result: ' + json.dumps(start_topics_detection_job_result))
while True:
    status = comprehend.list_topics_detection_jobs(
        Filter={
            'JobName': JobName
        }
    )
    
    if status['TopicsDetectionJobPropertiesList'][0]['JobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)
print(status)

start_topics_detection_job_result: {"JobId": "769a6b15c4792303a82c8cb5d7ac5b86", "JobStatus": "SUBMITTED", "ResponseMetadata": {"RequestId": "6657bd8b-e770-424c-a661-f826947cc228", "HTTPStatusCode": 200, "HTTPHeaders": {"x-amzn-requestid": "6657bd8b-e770-424c-a661-f826947cc228", "content-type": "application/x-amz-json-1.1", "content-length": "68", "date": "Mon, 02 Mar 2020 14:32:33 GMT"}, "RetryAttempts": 0}}
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready

In [130]:
job_id = start_topics_detection_job_result["JobId"]
print('job_id: ' + job_id)

def json_default(value): 
    import datetime, json
    if isinstance(value, datetime.date): 
        return value.strftime('%Y-%m-%d') 
    raise TypeError('not JSON serializable')
    
    
describe_topics_detection_job_result = comprehend.describe_topics_detection_job(JobId=job_id)
print('describe_topics_detection_job_result: ' + json.dumps(describe_topics_detection_job_result, 
                                                            default=json_default))

job_id: 769a6b15c4792303a82c8cb5d7ac5b86
describe_topics_detection_job_result: {"TopicsDetectionJobProperties": {"JobId": "769a6b15c4792303a82c8cb5d7ac5b86", "JobName": "test-topic-g", "JobStatus": "COMPLETED", "SubmitTime": "2020-03-02", "EndTime": "2020-03-02", "InputDataConfig": {"S3Uri": "s3://comprehend-dataset-cyj/topic", "InputFormat": "ONE_DOC_PER_FILE"}, "OutputDataConfig": {"S3Uri": "s3://comprehend-dataset-cyj/output/322537213286-TOPICS-769a6b15c4792303a82c8cb5d7ac5b86/output/output.tar.gz"}, "NumberOfTopics": 10, "DataAccessRoleArn": "arn:aws:iam::322537213286:role/service-role/AmazonComprehendServiceRole-comprehend-topic-model-cyj"}, "ResponseMetadata": {"RequestId": "91c8c9d4-b9ad-4af8-a9ec-264d15e2ff31", "HTTPStatusCode": 200, "HTTPHeaders": {"x-amzn-requestid": "91c8c9d4-b9ad-4af8-a9ec-264d15e2ff31", "content-type": "application/x-amz-json-1.1", "content-length": "560", "date": "Mon, 02 Mar 2020 15:14:01 GMT"}, "RetryAttempts": 0}}


In [121]:
res = describe_topics_detection_job_result['TopicsDetectionJobProperties']['OutputDataConfig']['S3Uri']
tmp=res.split('/')

In [122]:
bucket = tmp[2]
output_filename = tmp[3] +"/" + tmp[4] +"/" + tmp[5]+"/" + tmp[6]
output_path = './output/' + tmp[6]
print("bucket : {}, output_path : {}".format(bucket, output_path))

bucket : comprehend-dataset-cyj, output_path : ./output/output.tar.gz


In [123]:
s3.Object(bucket, output_filename).download_file(output_path)

In [124]:
ap = tarfile.open(output_path)
ap.extractall('./output/topic')
ap.close()

In [128]:
pd.read_csv('./output/topic/topic-terms.csv')

,topic,term,weight
0,0,거야,0.010164
1,0,오늘,0.008912
2,0,내,0.008411
3,0,거야,0.005798
4,0,오늘,0.005117
5,0,내,0.004796
6,0,다,0.004714
7,0,많이,0.004401
8,0,정말,0.003572
9,0,거,0.002919


In [127]:
pd.read_csv('./output/topic/doc-topics.csv')

,docname,topic,proportion
0,result-1493.txt,4,0.567149
1,result-1493.txt,0,0.238175
2,result-1493.txt,8,0.194676
3,result-1597.txt,5,0.648700
4,result-1597.txt,2,0.351300
5,result-1006.txt,8,1.000000
6,result-148.txt,4,1.000000
7,result-1205.txt,3,1.000000
8,result-1010.txt,2,0.503377
9,result-1010.txt,1,0.407446
